# Clustering Crypto

In [1]:
!pip install -U altair

     |████████████████████████████████| 727 kB 7.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [3]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from path import Path

### Fetching Cryptocurrency Data

In [4]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"


In [17]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.


In [18]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path, index_col='Unnamed: 0')
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [19]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
crypto_df = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [20]:
# Keep only cryptocurrencies that are trading
crypto_df = crypto_df[crypto_df['IsTrading'] == True]
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [21]:
# Keep only cryptocurrencies with a working algorithm
crypto_df= crypto_df.dropna(subset=['Algorithm'])
crypto_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [22]:
# Remove the 'IsTrading' column
crypto_df = crypto_df.drop(columns=['IsTrading'])
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0


In [23]:
# Remove rows with at least 1 null value
crypto_df.dropna(inplace=True)
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [24]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df[crypto_df['TotalCoinsMined']!=0]
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [25]:
# Drop rows where there are 'N/A' text values
for col in crypto_df.columns:
    crypto_df = crypto_df[crypto_df[col] != 'N/A']
crypto_df.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [26]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coins_name = crypto_df['CoinName']
coins_name.index = crypto_df.index

In [27]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns=['CoinName'], inplace = True)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [28]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X.head()


,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [30]:
# Use PCA to reduce dimensions to 3 principal components
# Initialize PCA model
pca = PCA(n_components=3)

# Get two principal components for the data.
X_pca = pca.fit_transform(X_scaled)



In [31]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=X_pca, columns=["PC 1", "PC 2", "PC 3"]
)
pcs_df.index = crypto_df.index
pcs_df.head(10)


,PC 1,PC 2,PC 3
42,-0.324161,1.045762,-0.650358
404,-0.307479,1.046007,-0.650871
1337,2.312146,1.629001,-0.745360
BTC,-0.144727,-1.304227,0.231658
ETH,-0.159160,-2.064928,0.392820
LTC,-0.156078,-1.078307,-0.009445
DASH,-0.395627,1.134437,-0.554927
XMR,-0.153632,-2.305566,0.440129
ETC,-0.157601,-2.065018,0.392804
ZEC,-0.149149,-2.061693,0.257546


### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [40]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
alt.Chart(df_elbow).mark_line().encode(x="k", y="inertia").properties(title = "Elbow Curve")



alt.Chart(...)

Running K-Means with `k=4`

In [46]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(pcs_df)
# Predict clusters
predictions = model.predict(pcs_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.concat([crypto_df, pcs_df, coins_name], axis=1, join='inner')
clustered_df["class"] = model.labels_
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.324161,1.045762,-0.650358,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.307479,1.046007,-0.650871,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.312146,1.629001,-0.745360,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144727,-1.304227,0.231658,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.159160,-2.064928,0.392820,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.156078,-1.078307,-0.009445,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.395627,1.134437,-0.554927,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.153632,-2.305566,0.440129,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.157601,-2.065018,0.392804,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.149149,-2.061693,0.257546,ZCash,1


### Visualizing Results

#### 3D-Scatter with Clusters

In [54]:
# Create a 3D-Scatter with the PCA data and the clusters

alt.Chart(clustered_df).mark_circle(size=60).encode(
    x='PC 1',
    y='PC 2',
    color='class',
    tooltip=['CoinName', 'Algorithm',  'TotalCoinsMined', 'TotalCoinSupply']
).interactive()

alt.Chart(...)

#### Table of Tradable Cryptocurrencies

In [64]:
# Table with tradable cryptos
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    display(clustered_df)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.324161,1.045762,-0.650358,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.307479,1.046007,-0.650871,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.312146,1.629001,-0.745360,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.144727,-1.304227,0.231658,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.159160,-2.064928,0.392820,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.156078,-1.078307,-0.009445,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.395627,1.134437,-0.554927,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.153632,-2.305566,0.440129,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.157601,-2.065018,0.392804,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.149149,-2.061693,0.257546,ZCash,1


In [63]:
# Print the total number of tradable cryptocurrencies
number_of_tradable_crypto = len(clustered_df.index)
print(number_of_tradable_crypto)

533


#### Scatter Plot with Tradable Cryptocurrencies

In [65]:
# Scale data to create the scatter plot
cluster_scaled = StandardScaler().fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
cluster_scaled =  pd.DataFrame(
    data=cluster_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined']
)
cluster_scaled['CoinName'] = coins_name.values
cluster_scaled.head()

,TotalCoinSupply,TotalCoinsMined,CoinName
0,-0.152865,-0.116748,42 Coin
1,-0.144996,-0.093589,404Coin
2,4.493764,0.525872,EliteCoin
3,-0.152554,-0.116354,Bitcoin
4,-0.152865,-0.114384,Ethereum


In [69]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
alt.Chart(cluster_scaled).mark_circle(size=60).encode(
    x="TotalCoinsMined", 
    y="TotalCoinSupply",
    tooltip=['CoinName']
    ).interactive()



alt.Chart(...)